https://sunitach10.github.io/2019/09/10/DeepDT-in-pytorch.html
    
https://academic.oup.com/bioinformatics/article/39/2/btad049/6998204

In [9]:
#find sequence
import requests
import pandas as pd

df = pd.read_csv('data/AMP_SMILES_FROM_FILE.csv', sep =';' )

err_prots=[]
for i in range(len(df)):
    code = df.loc[i, 'PDB code']
    try:
        data = requests.get(f'https://www.ebi.ac.uk/pdbe/api/pdb/entry/molecules/{code}').json()[code.lower()]
        df.loc[i, 'molecule_type'] = data[0]['molecule_type']
        df.loc[i, 'seq']      = data[0]['sequence']    
    except: 
        err_prots.append(df.loc[i, 'PDB_code'])
        pass 

print('errors ', err_prots)


df.to_csv("data/AMP_SEQUENCE_FROM_PDB.csv")


errors  []


In [ ]:
# mol2
from rdkit import Chem
import os

path = 'data/AMP/2020'

for x in os.listdir(path):
    if x.endswith(".mol2"):
        mol2_file = path + '/' + x
        print (mol2_file)
        m = Chem.rdmolfiles.MolFromMol2File(mol2_file)
        smi_isomeric  = Chem.MolToSmiles(m)
        print(smi_isomeric) 
        #print(smi_canonical)

#print(smi_isomeric_sdf) 
#print(smi_canonical_sdf)



In [ ]:
#SDF
sdf_file ='data/3i4y_ligand.sdf'
mols = [mol for mol in Chem.SDMolSupplier(sdf_file)]
for mol in mols:
    smi = Chem.MolToSmiles(mol)
    name = mol.GetProp("_Name")
    print(smi)
    print(name)


    

In [ ]:
#input a compound in SMILES format where not exist in training or validation or test dataset
#1. create the csv file with couple new coumpound and targets (previous procedure)
#2. predict affinity for all couples
#3. order by affinity desc
# test for compound 13-mer




In [17]:
#*************************  WORKING ****************************************
#                          HYBRID
# input is the 256 from embeding
# mporei na thelei  dilaSize [1,2,4,8,16]
#sto dilation DEN xreiazetai maxpool
#1 . na prosthew kai ta apla stoixeia
#2. na auxisw ta epipeda

import torch
import torch.nn as nn
from torch.nn import Sequential, Linear, ReLU
import math
import pennylane as qml
from functools import partial
import pennylane.numpy as np


#mporei na perasei san parameteros ston init ths klasis
n_qubits  = 8
n_layers  = 3 # 3 gia ola 1 gia block to paradeigma  PennyLane_01 eixa 6
n_features = 256
#n_angle_encoding = math.ceil(((n_features)/n_qubits)/2) #dense angle
n_angle_encoding = math.ceil(((n_features)/n_qubits)) #angle
n_blocks = 2

# default.qubit RUN ALL EXPERIMENTS**********
dev = qml.device("default.qubit", wires=n_qubits )
#dev = qml.device("lightning.qubit", wires=n_qubits )  # NOT USED*************

#   NOISE SIMULATORS************************************8
# Describe noise
#noise_gate = qml.AmplitudeDamping
#noise_gate = qml.DepolarizingChannel
#noise_strength = 0.1

# Load devices
#dev_ideal = qml.device("default.qubit, wires=n_qubits)  # the ideal device with no noise
#dev_noisy = qml.transforms.insert(noise_gate, noise_strength)(dev_ideal) # the noisy device with noise
#dev_noisy = qml.transforms.insert(dev_ideal, noise_gate, noise_strength, position="all")



#  *********************************************
# NOT USED *********************************
#   *********************************************
@qml.qnode(dev)
def quantum_state(fetaures, n_qubits):
     qml.AmplitudeEmbedding(fetaures, wires=range(n_qubits))

def layer(layer_weights, n_qubits):
    for i  in range(n_qubits):
        qml.Rot(*layer_weights[i], wires=i)

    for i in range(n_qubits):
        qml.CNOT(wires=[i, (i + 1) % n_qubits])      
     

def circuit_old(weights, features, n_qubits):
    quantum_state(features, n_qubits)

    for W in weights:
        layer(W, n_qubits)    
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev)
def circuit_amplitude(inputs, weights):                 #inputs  256 (batch size) x 256  output previous layer

    #print('weights.shape', weights.shape)
    #print('weights', weights)

    qml.AmplitudeEmbedding(inputs, wires=range(n_qubits), normalize=True )

    #print('inputs.shape', inputs.shape)
    #print('inputs', inputs)

    for W in weights: #->  LAYER  ************mipws prepei na ginei    for l in range(n_layers): einai to layer
        for i in range(n_qubits):
            qml.Rot(*W[i], wires=i)                                   #qml.Rot(weights[l, i + (n_qubits - 1)], wires=i) 

        for i in range(n_qubits):
            qml.CNOT(wires=[i, (i + 1) % n_qubits])

    # prosthesa CNOT  me 2 orisma to qubit 1 22/4/2024
    for i in range(n_qubits-1):
         qml.CNOT(wires=[ (i + 1) % n_qubits, 0])
         
    return  qml.expval(qml.PauliZ(0))


#  *********************************************
#  **********************************************
#  *********************************************


@qml.qnode(dev)
def circuit_amplitude_qml(inputs, weights):
    #print('inputs.shape', inputs.shape)
    #print('inputs', inputs)

    qml.AmplitudeEmbedding(inputs, wires=range(n_qubits), normalize=True)
    qml.StronglyEntanglingLayers(weights, wires = range (n_qubits), ranges=[1,1,1])   

    # prosthesa CNOT  me 2 orisma to qubit 1 22/4/2024
    for i in range(n_qubits-1):
         qml.CNOT(wires=[ (i + 1) % n_qubits, 0])

    return qml.expval(qml.PauliZ(0))


# einai isodinamo me to panw mono poy to cnot = range[1,2,3] ενώ το πάνω είναι range[1,1,1]
@qml.qnode(dev)
def circuit_angle_qml(inputs, weights):
    #print('inputs.shape', inputs.shape)
    #print('inputs', inputs)
    for i in range(n_angle_encoding):
        qml.AngleEmbedding(math.pi * inputs[:,i*n_qubits:i*n_qubits+n_qubits], wires=range(n_qubits))    #rotation='X'
        qml.StronglyEntanglingLayers(weights[i], wires = range (n_qubits), ranges=[1,1,1])   

    # prosthesa CNOT  me 2 orisma to qubit 1 22/4/2024
    for i in range(n_qubits-1):
         qml.CNOT(wires=[ (i + 1) % n_qubits, 0])

    return qml.expval(qml.PauliZ(0))


# ta dedomena apo ta klasika layer prepei na  ginoun normalize
#@qml.qnode(dev)
@qml.qnode(dev)
def circuit_dense_angle(inputs, weights):                 #inputs  256 (batch size) x 256  output previous layer
    
    #print('inputs.shape', inputs.shape)
   # print('inputs', inputs)

    #print('weights.shape', weights.shape)
    #print('weights', weights)
    

    #2,3,8,3
   # for k in range(inputs.shape[0])
    for j in range(n_angle_encoding):               
        #print('j' , j)
        for i in range(n_qubits):                               #eencoding
            qml.RX(math.pi *  inputs[:, i*2   + n_qubits *j] ,  wires=i)      #dense endocoding kai oxi aplo
            qml.RY(math.pi *  inputs[:, i*2+1 + n_qubits *j],   wires=i)

            #for W in weights[j]:                #LAYER
            #    for i in range(n_qubits):
            #        qml.Rot(*W[i], wires=i)                                   #qml.Rot(weights[l, i + (n_qubits - 1)], wires=i) 

            #    for i in range(n_qubits):
            #        qml.CNOT(wires=[i, (i + 1) % n_qubits])
        
        qml.StronglyEntanglingLayers(weights[j], wires = range ( n_qubits ), ranges=[1,1,1])   

        # prosthesa CNOT  me 2 orisma to qubit 1 22/4/2024

    for i in range(n_qubits-1):
         qml.CNOT(wires=[ (i + 1) % n_qubits, 0])
            
    return qml.expval(qml.PauliZ(0))


#########   ALLAXA STO embed_dim 256 -> 128
class HQ_Net(nn.Module):
    
    def __init__(self, embed_dim=128,  n_filters=16, output_dim=128, dropout=0.2,  n_output=1, dilaSize=1, normalize=True):
        
        
        super(HQ_Net, self).__init__()

        self.embed_smile = nn.Embedding(65, embed_dim)
        self.embed_prot = nn.Embedding(26, embed_dim)
        
        #smiles 
        self.smiles = nn.Sequential(
            nn.Conv1d(in_channels= embed_dim, out_channels= embed_dim, kernel_size=3,padding=dilaSize, dilation=dilaSize),
            nn.ReLU(),
            nn.Conv1d(in_channels= embed_dim , out_channels= embed_dim, kernel_size=3,padding=dilaSize * 2 ,dilation=dilaSize * 2),
            nn.ReLU(),
            nn.Conv1d(in_channels= embed_dim , out_channels= embed_dim, kernel_size=3,padding=dilaSize * 4 ,dilation=dilaSize * 4),
            nn.ReLU(),
            nn.Conv1d(in_channels= embed_dim , out_channels= embed_dim, kernel_size=3,padding=dilaSize * 8 ,dilation=dilaSize * 8),
            nn.ReLU(),
            nn.Conv1d(in_channels= embed_dim , out_channels= embed_dim, kernel_size=3,padding=dilaSize * 16,dilation=dilaSize * 16),
            #nn.ReLU(), 7/4/2024 gia na epistrepsei 0 eww 1
            nn.Sigmoid(), 
            nn.AdaptiveMaxPool1d(1)                        
        )
            
        #proteins sequence
        self.proteins = nn.Sequential(
            nn.Conv1d(in_channels=embed_dim, out_channels=embed_dim, kernel_size=3,padding=dilaSize ,dilation=dilaSize),
            nn.ReLU(),
            nn.Conv1d(in_channels= embed_dim  , out_channels=embed_dim, kernel_size=3,padding=dilaSize *2 ,dilation=dilaSize *2),
            nn.ReLU(),
            nn.Conv1d(in_channels= embed_dim , out_channels= embed_dim, kernel_size=3,padding=dilaSize * 4 ,dilation=dilaSize * 4),
            nn.ReLU(),
            nn.Conv1d(in_channels= embed_dim , out_channels= embed_dim, kernel_size=3,padding=dilaSize * 8 ,dilation=dilaSize * 8),
            nn.ReLU(),
            nn.Conv1d(in_channels= embed_dim , out_channels= embed_dim, kernel_size=3,padding=dilaSize * 16,dilation=dilaSize * 16),
            #nn.ReLU(), 7/4/2024 gia na epistrepsei 0 eww 1
            nn.Sigmoid(),   
            nn.AdaptiveMaxPool1d(1),
          )
        

        #self.smiles_descriptors =  nn.Sequential(
        #     nn.Linear(6 , 6),
        #     nn.ReLU()
        # )
            
        #self.predict = nn.Sequential(
        #                            #nn.Linear(2 * embed_dim + 6, 1024), #6 smiles descriptors                                    #nn.Linear(2 * embed_dim + 6, 1024), #6 smiles descriptors
        #                            nn.Linear(2 * embed_dim , 1024), #6 smiles descriptors
        #                            nn.ReLU(),
        #                            nn.Dropout(dropout),
        #                            nn.Linear(1024, 512),
        #                            nn.ReLU(),
         #                           nn.Dropout(dropout),
         #                           nn.Linear(512, n_output)
         #                           )
      
        #for amplitube
        #weight_shapes = {"weights": ( n_layers, n_qubits, 3)}

        #for amplitube block
        #weight_shapes = {"weights": ( n_blocks,  n_layers, n_qubits, 3)}


        # for angle
        weight_shapes = {"weights": (n_angle_encoding, n_layers, n_qubits, 3)} # 3 einai gia to rot

       
        #qnode = qml.QNode(circuit_amplitude_qml, dev, interface='torch', diff_method='backprop')
        #qnode = qml.QNode(circuit_angle_qml, dev, interface='torch', diff_method='backprop')
        qnode = qml.QNode( circuit_angle_qml, dev, interface='torch', diff_method='backprop')

                
       # self.predict_q = qml.qnn.TorchLayer(qnode, weight_shapes, init_method=nn.init.uniform_) #Semantic
        
        self.predict_q = qml.qnn.TorchLayer(qnode, weight_shapes) 
        

        #nn.init.uniform_(self.predict_q.weight.data, a=0.0, b=0.01, generator=None)
  
    def forward(self, smi_in, seq_in,  smi_desc):#65,26

        #print('1. smi_in', smi_in.shape)        
        #print('1. seq_in', seq_in.shape)
        
        embedded_smi  = self.embed_smile(smi_in) 
        embedded_seq  = self.embed_prot(seq_in)
        
        #smi_desc =  self.smiles_descriptors(smi_desc) 
        
        #print('2. embedded_smi.shape', embedded_smi.shape)        
        #print('2. embedded_smi.shape', embedded_seq.shape)
        
        smi = self.smiles(embedded_smi.transpose(1,2))
        seq = self.proteins(embedded_seq.transpose(1,2))
        
        #print('3. smi.shape', smi.shape)        
        #print('3. seq.shape', seq.shape)
        
        #Squeeze = Reduce('b c t -> b c', 'max')(smi) #einai swsto?? diafora me Test_3
        #seq = Reduce('b c t -> b c', 'max')(seq) #einai swsto??
        
        smi = smi.squeeze()
        seq = seq.squeeze()

        #print('4. smi.shape', smi.shape)        
        #print('4. seq.shape', seq.shape)
        
        #smi = smi.view(-1, 71 * 128 * 26) #32 = n_filters * 2
        #seq = seq.view(-1, 71 * 128 * 26) #32 = n_filters * 2
        

        # concat
        #smi_seq = torch.cat((smi, seq, smi_desc),1)
        #print('smi.shape', smi.shape)
        #print('smi', smi )
        
        smi_seq =torch.cat((smi, seq) , 1)  # ΔΕΝ ΒΑΖΩ ΤΑ 6 χαρακτηριστικά

        #print('5.smi_seq ', smi_seq.shape)
        
        #PROSOXI GINETAI NORMALIZE EINAI MONO GIA AMPLIRUDE??????????????????????
        #smi_seq = smi_seq / np.linalg.norm(smi_seq)

        #shape = qml.math.shape(smi_seq)
        #norm = qml.math.sum(qml.math.abs(smi_seq) ** 2, axis=-1)
        #smi_seq = smi_seq / qml.math.reshape(qml.math.sqrt(norm), (*shape[:-1], 1))
        

        #out = self.predict(smi_seq)
        out = self.predict_q(smi_seq)            #quantum
        
        return out 

In [19]:
import torch

#file_path = 'models/HQ_4_qubit_dense_angle.pt'
file_path = 'models/HQ_8_qubit_angle.pt'

model = HQ_Net()
model.load_state_dict(torch.load(file_path))
model.eval()



C:\Users\Μαρία\AppData\Local\Temp\ipykernel_13392\1622923308.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(file_path))


HQ_Net(
  (embed_smile): Embedding(65, 128)
  (embed_prot): Embedding(26, 128)
  (smiles): Sequential(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (3): ReLU()
    (4): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (5): ReLU()
    (6): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
    (7): ReLU()
    (8): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
    (9): Sigmoid()
    (10): AdaptiveMaxPool1d(output_size=1)
  )
  (proteins): Sequential(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (3): ReLU()
    (4): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (5): ReLU()
    (6): Conv1d(128, 128, kernel_

In [23]:
from dataset import MyDataset_predict
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import pandas as pd
from pathlib import Path

batch_size = 256
data_path = 'data/' 
#format files ,PDB code,smiles,molecule_type,seq
file_names = ['AMP_SMILES_FROM_FILE',
              'AMP_SMILES_FROM_FILE_SEQ_FROM_PDB',
              'AMP_SMILES_FROM_PUBCHEM',
              'AMP_SMILES_FROM_PUBCHEM_SEQ_FROM_PDB',
              'AMP_SMILES_FROM_RCS',
              'AMP_SMILES_FROM_RCS_SEQ_FROM_PDB']

for file_name in  file_names:
    max_seq_len = 1000
    max_smi_len = 160
    
    test_loader =  DataLoader(MyDataset_predict(data_path, file_name, max_seq_len, max_smi_len),
                                             batch_size=batch_size,
                                             pin_memory=True,
                                             num_workers=4,
                                             shuffle= False)
    
    cuda_name = "cuda:0"
    device = torch.device(cuda_name if torch.cuda.is_available() else "cpu")
    
    outputs = []
    with torch.no_grad():
        for idx, (*x, y) in tqdm(enumerate(test_loader), disable=False, total=len(test_loader)):
            for i in range(len(x)):
                x[i] = x[i].to(device)
       
            y_hat = model(*x)
            outputs.append(y_hat.cpu().numpy().reshape(-1))
    
    
    outputs = np.concatenate(outputs).reshape(-1)
    
    
    print(outputs)
    
    
    # convert array into dataframe 
    df_o = pd.DataFrame(outputs) 
    output_path = Path('predict/')
    df_o.to_csv(output_path / f"{file_name}.csv")


  0%|          | 0/1 [00:00<?, ?it/s]

[0.32448918 0.34110376 0.2893315  0.33365691 0.25950497 0.289381
 0.31617552 0.32676232 0.32087007 0.31819445 0.32111076 0.29779857
 0.26789835 0.31664518 0.31266719 0.34390098 0.26368973 0.32454661
 0.30962488 0.32381776 0.29514199 0.32858428 0.29902297 0.32692522
 0.28144416 0.27950981 0.32877833 0.33919424 0.29251918 0.3388814 ]


  0%|          | 0/1 [00:00<?, ?it/s]

[0.32041615 0.3406339  0.28499755 0.33325028 0.25950497 0.289381
 0.31797197 0.32921922 0.31363738 0.31819445 0.32479626 0.30792978
 0.26789835 0.32434723 0.31390736 0.35254797 0.26386014 0.33773538
 0.31475815 0.32525966 0.29640913 0.32256579 0.30435574 0.32692522
 0.28470322 0.28470322 0.32738286 0.34615391 0.30389899 0.32879233]


  0%|          | 0/1 [00:00<?, ?it/s]

...

[0.36365426 0.3837842  0.32613871 0.3754355  0.30500656 0.33407617
 0.3616097  0.37211636 0.35679039 0.36211056 0.36880589 0.34987587
 0.31296951 0.3671883  0.35639399 0.39370355 0.30913687 0.38067922
 0.35821208 0.36913848 0.33967626 0.36691767 0.34890836 0.37077102
 0.32587647 0.32587647 0.36994699 0.38637164 0.34838429 0.37221971]


  0%|          | 0/1 [00:00<?, ?it/s]

[0.37614852 0.39258289 0.33935845 0.38352454 0.31330431 0.34139237
 0.3674202  0.37728074 0.37230685 0.3690736  0.37193379 0.34930202
 0.32131714 0.36827272 0.36472303 0.39422476 0.31715333 0.37587169
 0.36168751 0.37578583 0.34825194 0.37839112 0.35112706 0.37826833
 0.33101845 0.32960489 0.37990943 0.38876984 0.34525126 0.39051896]


  0%|          | 0/1 [00:00<?, ?it/s]

[0.37146857 0.39217988 0.33535638 0.38304397 0.31330431 0.34139237
 0.36986443 0.37969428 0.36531976 0.3690736  0.37634006 0.35866302
 0.32131714 0.37607858 0.36494058 0.4028599  0.31737021 0.38879958
 0.3662571  0.37706035 0.34862861 0.37351981 0.35614252 0.37826833
 0.33506939 0.33506939 0.37878853 0.39555424 0.3561506  0.37991557]
